In [2]:
import pandas as pd
import numpy as np
import os
import cvxpy as cp
pd.set_option('display.float_format', lambda x: '%.6f' % x)

def var_historic(r, level=5):
    '''
    Returns the Historic VaR at a specified level
    .i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100 - level) percent are above
    '''
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -1 * np.percentile(r, level, method='closest_observation')
    else:
        raise TypeError("Expected r to be Series or DataFrame")


def cvar_historic(r, level=5):
    '''
    Computes the conditional VaR of Series or DataFrame
    '''
    if isinstance(r, pd.Series):
        is_beyond = r <= -var_historic(r, level=level)
        return -r[is_beyond].mean()
    elif isinstance(r, pd.DataFrame):
        return r.aggregate(cvar_historic, level=level)
    else:
        raise TypeError("Expected r to be Series or DataFrame")


In [3]:
script_dr = os.getcwd()
df = pd.read_excel(os.path.abspath(os.path.join(script_dr, '..', 'Data', 'FERM_3_Assignment_2.xlsx')), sheet_name='Daily_Returns_Raw_Data').set_index('date')
sample_mean = df.mean().to_numpy()
sample_mean
mean_of_sample_mean = np.mean(sample_mean)

sample_vol = df.std().to_numpy()
sample_variance = sample_vol**2
mean_of_sample_variance = np.mean(sample_variance)
print(sample_mean, mean_of_sample_mean, mean_of_sample_variance)
df

[ 2.40820912e-04 -1.12599337e-03  3.78583479e-05 -2.23075471e-04
  1.16724913e-03 -8.06973529e-04  1.65834596e-03  1.47265620e-04
  6.08064560e-04  5.78082116e-03  2.69585629e-03  2.49879047e-04
 -8.14852156e-04  2.46768990e-04  1.90221404e-03  8.24732074e-04
 -5.68808453e-04  1.77670286e-03  7.40452501e-04  4.35394234e-04
 -1.78658878e-04  2.72623422e-04  6.13614239e-04] 0.0006817522404183187 0.0004595265359481587


,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,0.046629,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
2016-01-05,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,-0.006857,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2016-01-06,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,-0.061485,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
2016-01-07,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,0.026476,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
2016-01-08,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,-0.002378,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,0.006579,0.001716,-0.006328,-0.003967,0.002010,0.025091,-0.006211,0.007981,0.018798,0.030441,...,0.001542,0.004773,0.001596,-0.016616,-0.001816,0.000519,0.002776,-0.008819,-0.000974,-0.004051
2017-12-22,0.002162,-0.004008,-0.000941,0.001589,-0.003352,-0.016057,0.004662,0.011067,0.009268,0.082888,...,0.003077,0.039678,0.012678,0.003671,-0.007754,-0.001817,-0.000832,0.004909,-0.006847,-0.010201
2017-12-26,0.001961,-0.002873,0.001763,-0.007968,-0.001680,-0.003243,-0.006221,-0.003150,-0.005550,0.033523,...,-0.004362,0.001524,0.003145,-0.012565,0.009569,0.002077,0.004844,0.006833,-0.002949,-0.002738


In [4]:
cov = df.cov()
cov

,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
IBM,0.000127,0.000047,0.000019,0.000055,0.000070,0.000005,0.000073,0.000011,0.000032,0.000031,...,0.000024,0.000084,0.000046,0.000020,0.000058,0.000047,0.000016,0.000014,0.000050,0.000069
GE,0.000047,0.000139,0.000022,0.000070,0.000087,0.000015,0.000084,-0.000026,0.000039,-0.000014,...,0.000039,0.000085,0.000048,0.000039,0.000051,0.000041,0.000020,0.000032,0.000062,0.000076
LLY,0.000019,0.000022,0.000163,0.000030,0.000041,0.000020,0.000030,0.000011,0.000026,0.000011,...,0.000020,0.000019,0.000027,0.000029,0.000025,0.000034,0.000014,0.000071,0.000031,0.000030
F,0.000055,0.000070,0.000030,0.000204,0.000127,0.000035,0.000092,-0.000004,0.000071,0.000026,...,0.000032,0.000152,0.000051,0.000033,0.000089,0.000065,0.000011,0.000055,0.000094,0.000115
BAC,0.000070,0.000087,0.000041,0.000127,0.000296,0.000058,0.000129,-0.000121,0.000085,0.000044,...,0.000019,0.000214,0.000072,0.000045,0.000103,0.000069,-0.000009,0.000040,0.000175,0.000233
KR,0.000005,0.000015,0.000020,0.000035,0.000058,0.000366,-0.000016,-0.000024,0.000030,0.000042,...,0.000015,0.000049,0.000016,0.000063,0.000038,0.000026,0.000009,0.000028,0.000042,0.000043
ITUB,0.000073,0.000084,0.000030,0.000092,0.000129,-0.000016,0.000701,0.000130,0.000070,0.000099,...,0.000052,0.000294,0.000340,0.000038,0.000127,0.000113,0.000050,0.000063,0.000068,0.000106
GG,0.000011,-0.000026,0.000011,-0.000004,-0.000121,-0.000024,0.000130,0.000642,-0.000003,-0.000058,...,0.000030,0.000148,0.000090,-0.000015,0.000009,0.000014,0.000012,0.000016,-0.000079,-0.000110
SIRI,0.000032,0.000039,0.000026,0.000071,0.000085,0.000030,0.000070,-0.000003,0.000169,0.000052,...,0.000029,0.000094,0.000048,0.000020,0.000076,0.000047,0.000019,0.000018,0.000057,0.000063
ACB,0.000031,-0.000014,0.000011,0.000026,0.000044,0.000042,0.000099,-0.000058,0.000052,0.002376,...,-0.000006,0.000149,0.000103,0.000036,0.000066,0.000045,0.000019,-0.000018,0.000025,0.000057


In [5]:
alpha = 0.9
I = np.identity(len(sample_mean))
est_variance = alpha * cov + (1-alpha)*I*mean_of_sample_variance
est_variance


,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
IBM,0.000160,0.000042,0.000017,0.000050,0.000063,0.000005,0.000066,0.000010,0.000028,0.000028,...,0.000021,0.000075,0.000042,0.000018,0.000052,0.000043,0.000014,0.000013,0.000045,0.000062
GE,0.000042,0.000171,0.000019,0.000063,0.000078,0.000014,0.000076,-0.000023,0.000035,-0.000013,...,0.000035,0.000077,0.000043,0.000035,0.000046,0.000037,0.000018,0.000029,0.000056,0.000069
LLY,0.000017,0.000019,0.000193,0.000027,0.000037,0.000018,0.000027,0.000010,0.000024,0.000010,...,0.000018,0.000017,0.000025,0.000026,0.000022,0.000030,0.000013,0.000063,0.000028,0.000027
F,0.000050,0.000063,0.000027,0.000229,0.000114,0.000031,0.000083,-0.000003,0.000064,0.000023,...,0.000029,0.000137,0.000045,0.000030,0.000080,0.000058,0.000010,0.000050,0.000084,0.000103
BAC,0.000063,0.000078,0.000037,0.000114,0.000312,0.000052,0.000116,-0.000109,0.000077,0.000040,...,0.000017,0.000193,0.000065,0.000040,0.000092,0.000062,-0.000008,0.000036,0.000158,0.000210
KR,0.000005,0.000014,0.000018,0.000031,0.000052,0.000375,-0.000014,-0.000022,0.000027,0.000038,...,0.000013,0.000044,0.000014,0.000057,0.000034,0.000023,0.000008,0.000025,0.000038,0.000038
ITUB,0.000066,0.000076,0.000027,0.000083,0.000116,-0.000014,0.000677,0.000117,0.000063,0.000089,...,0.000047,0.000265,0.000306,0.000034,0.000115,0.000101,0.000045,0.000057,0.000061,0.000095
GG,0.000010,-0.000023,0.000010,-0.000003,-0.000109,-0.000022,0.000117,0.000624,-0.000003,-0.000052,...,0.000027,0.000134,0.000081,-0.000013,0.000008,0.000013,0.000011,0.000015,-0.000071,-0.000099
SIRI,0.000028,0.000035,0.000024,0.000064,0.000077,0.000027,0.000063,-0.000003,0.000198,0.000047,...,0.000026,0.000084,0.000043,0.000018,0.000068,0.000043,0.000017,0.000016,0.000052,0.000057
ACB,0.000028,-0.000013,0.000010,0.000023,0.000040,0.000038,0.000089,-0.000052,0.000047,0.002185,...,-0.000006,0.000134,0.000093,0.000032,0.000059,0.000041,0.000017,-0.000017,0.000023,0.000051


### Problem 1
There are 23 stocks' daily returns from January 4th, 2016 to December 28th, 2017 from Yahoo Finance. Please download the 'FERM_3_Assignment_2.xlsx' file attached.
Now we want to construct the mean-variance portfolio in a more practical way.
Compute the estimation of returns by shrinkage method:
$\mu_i = \alpha R_i + (1-\alpha)(\frac{1}{23} \sum_{i=1}^{23} R_i$
$V^{est} = \alpha V + (1-\alpha)(\frac{1}{23} (\sum_{i=1}^{23} \sigma_{i}^{2})I$
Where $R_i$ is $\sigma_{i}^{2}$ are sample mean and sample variance for the $i^{th}$ asset, respectively. Compute the value of the $\mu_1$(for IBM) when $\alpha=0.9$. (Round to the fourth decimal place.)
Submission Guideline: Give your answer in rounded to 4 decimal places. For example, if you compute the answer to be 2.67%, submit 0.0267.

In [6]:
est_mean = alpha * sample_mean + (1-alpha)*mean_of_sample_mean
print(f"The IBM mean is {np.round(est_mean[0], 4)}")

The IBM mean is 0.0003


### Problem 2
Suppose we want to minimize the risk of the portfolio with the constraint that average daily return should be greater than or equal to 0.05%. i.e. :
$minimise x^{T} V x$
$s.t. \mu^{T} x >= 0.0005$
$1^T x = 1 $
Where $\mu$ is the daily return vector of the stocks, x is the weight vector of stocks in the portfolio, and V is the covariance matrix of the stocks' daily returns.
Now we want to add a constraint that no short is allowed, i.e. $x≥0$ should be satisfied. Compute the percent of wealth invested in the stock IBM. (Round to the third decimal place.)
Submission Guideline: Give your answer in rounded to 3 decimal places. For example, if you compute the answer to be 2.6%, submit 0.026.

In [7]:
n = len(sample_mean)
w = cp.Variable(n)
ret = est_mean.T @ w
risk = (w.T @ est_variance @ w)
prob = cp.Problem(cp.Minimize(risk), [cp.sum(w) == 1, w>=0, ret>=0.05/100 ])
result = prob.solve()

w.value
# ret.value
print(f"The wealth of IBM stock is {np.round(w.value[0], 3)}")

The wealth of IBM stock is 0.081


### Problem 3
Continued from Question 2

Suppose we change the no short constraint to one leverage constraint that $\sum_{i=0}^{23} <= M $ where $M=0.1$, then the problem becomes:
$minimise$ $ x^T$ $V$ $x$
$s.t.$  $\mu^T$  $x$ $>=$ $0.0005$
$1^T x = 1$
$\sum_{i=1}^{23} x_{i}^{-} <= 0.1 $
$x_{i}^{+} - x_{i}^{-} = x_i $
$x_{i}^{+} >= 0 , x_{i}^{-} >= 0$
where $x_{i}^{+}$ and $x_{i}^{-}$ are short and long position in asset $i$, respectively.
Compute the percent of wealth invested in the stock IBM. (Round to the third decimal place.)
Submission Guideline: Give your answer in rounded to 3 decimal places. For example, if you compute the answer to be 2.6%, submit 0.026.

In [8]:
n = len(sample_mean)
# Lmax = cp.Parameter()
w = cp.Variable(n)
w_long =  cp.Variable(n)
w_short = cp.Variable(n)

ret = est_mean.T @ w
risk = (w.T @ est_variance @ w)
prob = cp.Problem(cp.Minimize(risk), [cp.sum(w) == 1,  ret>=0.05/100 , w == w_long - w_short, cp.sum(w_short) <=0.1, w_long>=0, w_short>=0 ])
result = prob.solve()
# print(w.value, w_long.value, w_short.value)
# ret.value
# print(np.round(w.value[0], 3))

print(f"The wealth of IBM stock with short constraint is {np.round(w.value[0], 3)}")

The wealth of IBM stock with short constraint is 0.082


### Problem 4

Continued from Question 3

In the previous question, we want to draw the efficient frontier from $r=0.0003$ to $r=0.001$ by solving the following problem twice with $r_1 = 0.0003$ and $r_2=0.001$. Let $x_1$ and $x_2$ be the solutions, respectively.
$minimise$ $ x^T$ $V$ $x$
$s.t.$  $\mu^T$  $x$ $>=$ $0.0005$
$1^T x = 1$
$\sum_{i=1}^{23} x_{i}^{-} <= 0.1 $
$x_{i}^{+} - x_{i}^{-} = x_i $
$x_{i}^{+} >= 0 , x_{i}^{-} >= 0$
where $x_{i}^{+}$ and $x_{i}^{-}$ are short and long position in asset $i$, respectively.
Is $0.5x_1$ $+$ $0.5x_2$  still on the frontier?

In [9]:
print("yes")

yes


### Problem 5
Continued from Question 4

If we cannot invest more than 10% of our wealth in any one stock, which constraint should be used?

In [10]:
print("x_i <= 0.1")

x_i <= 0.1


## VaR and CVaR Optional Quiz

In [11]:
(np.std(df)**(2)).sort_values(ascending=False).index[0]

'ACB'

In [12]:
var_historic(df, level=10).sort_values(ascending=False).index[0]

'ACB'

In [13]:
cvar_historic(df, level=10).sort_values(ascending=False).index[0]

'ACB'

In [14]:
wt = np.ones(len(sample_mean)) * (1/(len(sample_mean)))
eq_wt_df = (1/23) * df
# var_historic(eq_wt_df, level=10)
# wt
portfolio_returns = eq_wt_df.sum(axis=1)
np.round(var_historic(portfolio_returns, level=10), 4)

0.009

In [15]:
wt = np.ones(len(sample_mean)) * (1/(len(sample_mean)))
c_cvar_90 = cvar_historic(df, level=10)
c_cvar_90.sort_values(ascending=False)
c_cvar_90
df
wt
np.round(cvar_historic(np.sum(wt * df, axis=1), level=10), 4)
# -0.000736 *0.043478


0.0158

In [16]:
np.round(cvar_historic(portfolio_returns, level=10), 4)

0.0158

In [17]:
pnl1= pd.Series(np.array([-0.012, 0.021, 0.0212, 0.0111, -0.0054, 0.0254, -0.0195, -0.003, 0.008, -0.021]))
pnl2 = pd.Series(np.array([-0.012, 0.021, 0.0212, 0.0111, -0.0054, 0.0254, -0.0195, -0.003, 0.008, -0.03]))
np.round((cvar_historic(pnl2, level=10) - cvar_historic(pnl1, level=10)), 4)


0.009

In [22]:
from scipy.optimize import minimize
def var_historic(r, level=5):
    '''
    Returns the Historic VaR at a specified level
    .i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100 - level) percent are above
    '''
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -1 * np.percentile(r, level, method='closest_observation')
    else:
        raise TypeError("Expected r to be Series or DataFrame")


def cvar_historic(r, level=5):
    '''
    Computes the conditional VaR of Series or DataFrame
    '''
    if isinstance(r, pd.Series):
        is_beyond = r <= -var_historic(r, level=level)
        return -r[is_beyond].mean()
    elif isinstance(r, pd.DataFrame):
        return r.aggregate(cvar_historic, level=level)
    else:
        raise TypeError("Expected r to be Series or DataFrame")

def portfolio_returns(weights, returns):
    '''
    Weights --> Returns
    '''
    # print(weights)
    # print(-1*(weights.T @ returns))
    return -1*(weights.T @ returns)

def cvar_risk(target_cvar: float, sample_returns: pd.DataFrame):
    rows, cols = sample_returns.shape
    er = sample_returns.mean()
    # print(er)
    init_guess = np.repeat(1 / cols, cols)
    bounds = ((0.0, 1.0),) * cols
    weights_sum_to_1 = {
        'type': 'eq',
        'fun': lambda weights: np.sum(weights) - 1
    }

    cvar_is_target = {
        'type': 'eq',
        'args' : (sample_returns, ),
        'fun': lambda weights, sample_returns: target_cvar - cvar_historic((weights * sample_returns).sum(axis=1), level=10)
    }
    results = minimize(portfolio_returns, init_guess, args=(er,), method='SLSQP',
                       options={'disp': False},
                       constraints=(cvar_is_target, weights_sum_to_1),
                       bounds=bounds
                       )
    return results.x

wts_cvar_90 = cvar_risk(target_cvar=0.02, sample_returns=df)
# print(f"The CVaR90 at gamma {0.02} return is {np.round(wts_cvar_90.T @ df.mean(), 6)}")
wts_cvar_90

array([4.73538956e-02, 0.00000000e+00, 2.11659905e-02, 0.00000000e+00,
       5.42051978e-02, 1.72794721e-19, 3.29264904e-02, 5.52891785e-02,
       6.07043291e-02, 2.14640316e-01, 5.79028462e-03, 6.75659461e-02,
       8.40256684e-19, 8.22735722e-02, 2.20608422e-02, 2.31073439e-02,
       4.30970364e-18, 1.35715431e-01, 5.91545745e-02, 1.11317347e-01,
       0.00000000e+00, 0.00000000e+00, 6.72926036e-03])

In [23]:
# udf = pd.read_csv('stock_returns_2018.csv')
from pypfopt.efficient_frontier import EfficientCVaR
from pypfopt import expected_returns

ec = EfficientCVaR(df.mean(), df, beta=0.9)
ec.efficient_risk(target_cvar=0.02)

OrderedDict([('IBM', 1.62928e-11),
             ('GE', 1.9478e-12),
             ('LLY', 8.5654e-12),
             ('F', 3.3918e-12),
             ('BAC', 0.0839600816334647),
             ('KR', 4.166e-12),
             ('ITUB', 0.0546320630016249),
             ('GG', 0.0769359947093318),
             ('SIRI', 9.61419e-11),
             ('ACB', 0.2067620508232054),
             ('VALE', 0.0244810744259999),
             ('PFE', 0.0179893691744805),
             ('CTL', 1.8851e-12),
             ('T', 0.1148234953683541),
             ('ECA', 1.01238e-11),
             ('ABEV', 1.27055e-11),
             ('CVS', 3.6544e-12),
             ('MRVL', 0.207811274641882),
             ('CSCO', 3.1067e-11),
             ('MO', 0.2126045960084068),
             ('BMY', 7.9196e-12),
             ('WFC', 6.3081e-12),
             ('HBAN', 9.0807e-12)])